In [1]:
import pandas as pd
import torch

In [2]:
tonight_odds = pd.read_excel('games_odds.xlsx', sheet_name='Tonight')
prev_odds = pd.read_excel('games_odds.xlsx', sheet_name='Previous')

In [3]:
import os
os.chdir('..')
from team_functions import dataPrep, getTeamRollingSeason, mapTeamID
from models import *

In [4]:
games_to_pred = []
teams, opps = tonight_odds['visitor'], tonight_odds['local']
for team, opp in zip(teams, opps):
    team_row = {}
    team_df = pd.read_excel(f'data/CurrentSeason/{team}.xlsx')
    
    team_row['Game'] = team_df['Game'].iloc[-1] + 1
    team_row['Date'] = tonight_odds.loc[tonight_odds['visitor'] == team, 'date'].iloc[0]
    team_row['Streak'] = team_df['Streak'].iloc[-1]

    team_cols = ['Pts', 'Pace', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in team_cols:
        team_row[col] = team_df[col].mean()

    opp_df = pd.read_excel(f'data/CurrentSeason/{opp}.xlsx')
    team_row['OppID'] = mapTeamID(opp)
    
    opp_cols = ['Pts', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in opp_cols:
        team_row['Opp'+col] = opp_df[col].mean()
        
    team_row['Location'] = 0
    
    team_row['Month'] = team_row['Date'].month
    team_row['DayOfWeek'] = team_row['Date'].weekday()
    team_row['DaysOfRest'] = (team_row['Date'] - team_df['Date'].iloc[-1]).days
    
    team_row['TeamID'] = mapTeamID(team)
    team_row['Season'] = 2024
    
    games_to_pred.append(team_row)
    
games_df = pd.DataFrame(games_to_pred)

In [5]:
games_df

,Game,Date,Streak,Pts,Pace,eFG,TOV,ORB,FTR,ORT,...,OppTOV,OppORB,OppFTR,OppORT,Location,Month,DayOfWeek,DaysOfRest,TeamID,Season
0,71,2024-03-27,-2,118.342857,99.022857,0.553957,11.828571,27.337143,0.179243,118.521429,...,12.736620,25.738028,0.220746,114.494366,0,3,2,3,10,2024
1,73,2024-03-27,1,112.680556,96.305556,0.556069,11.645833,23.316667,0.183236,116.209722,...,12.014085,21.384507,0.165056,109.504225,0,3,2,2,6,2024
2,73,2024-03-27,1,110.888889,96.229167,0.533486,11.129167,24.961111,0.177958,114.525000,...,11.888889,20.356944,0.171792,111.154167,0,3,2,2,3,2024
3,72,2024-03-27,-2,116.450704,96.326761,0.567648,11.301408,24.267606,0.216225,120.688732,...,10.037500,24.773611,0.225083,117.602778,0,3,2,2,13,2024
4,72,2024-03-27,2,111.943662,94.776056,0.536465,11.138028,29.364789,0.193944,118.130986,...,11.679167,24.341667,0.180083,113.666667,0,3,2,2,20,2024
5,73,2024-03-27,-7,107.583333,96.430556,0.508681,12.730556,27.020833,0.190069,110.361111,...,10.760563,27.608451,0.207408,119.176056,0,3,2,2,25,2024
6,73,2024-03-27,4,117.833333,100.054167,0.565236,11.787500,19.627778,0.215958,116.552778,...,12.847222,22.952778,0.188208,108.075000,0,3,2,1,14,2024
7,74,2024-03-27,1,123.260274,101.267123,0.580493,10.906849,23.253425,0.176479,121.547945,...,10.508333,24.245833,0.189694,115.526389,0,3,2,2,12,2024
8,72,2024-03-27,9,114.197183,98.402817,0.530648,10.809859,24.923944,0.203042,115.291549,...,10.474286,20.865714,0.202243,120.764286,0,3,2,2,11,2024
9,73,2024-03-27,-7,111.111111,99.481944,0.530444,13.152778,24.473611,0.198722,111.265278,...,12.312676,23.192958,0.211915,116.705634,0,3,2,2,9,2024


In [6]:
import joblib

In [7]:
def makeModelPredictions(games_df, odds_df, model_name='LR', nn_name='RNN',show=True):
    if model_name == 'NN':
        if nn_name == 'FCNN':
            fcnns = pd.read_excel('Models/TrainedModels/FCNNs/models.xlsx')
            fcnns = fcnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = fcnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        elif nn_name == 'RNN':
            rnns =  pd.read_excel('Models/TrainedModels/Recurrents/models.xlsx')
            rnns = rnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = rnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        
        scaler = joblib.load(f'Models/Scalers/{nn_name}.pkl')
        
    else:
        model = joblib.load(f'Models/TrainedModels/{model_name}_Model.pkl')
        scaler = joblib.load(f'Models/Scalers/{model_name}.bin')
        
    
    scaled_df = scaler.transform(games_df.drop(columns=['Date']))
    if model_name == 'NN':
        scaled_df = torch.FloatTensor(scaled_df)
        predictions = torch.round(model(scaled_df).squeeze(1))
        predictions = predictions.detach().numpy()
    else:
        predictions = model.predict(scaled_df)
    
    if model_name == 'NN':
        model_name = nn_name
        
    odds_df['Predictions'] = predictions
    if show:
        print(f'Predictions made by: {model_name}')
        display(odds_df)
        
    odds_df.to_excel(f'Predictions/ModelsPredictions/{model_name}.xlsx', index=0)

In [8]:
makeModelPredictions(games_df, tonight_odds, 'LR')

Predictions made by: LR


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-27,GSW,ORL,2.600000,1.520833,0
1,2024-03-27,CLE,CHO,1.192308,4.900000,1
2,2024-03-27,BRK,WAS,1.645161,2.300000,1
3,2024-03-27,LAC,PHI,1.408163,3.000000,0
4,2024-03-27,NYK,TOR,1.105263,7.250000,1
5,2024-03-27,POR,ATL,5.100000,1.181818,0
6,2024-03-27,LAL,MEM,1.571429,2.450000,1
7,2024-03-27,IND,CHI,1.704225,2.200000,1
8,2024-03-27,HOU,OKC,3.100000,1.387597,0
9,2024-03-27,DET,MIN,9.500000,1.068966,0


In [9]:
makeModelPredictions(games_df, tonight_odds, 'RF')

Predictions made by: RF


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-27,GSW,ORL,2.600000,1.520833,1
1,2024-03-27,CLE,CHO,1.192308,4.900000,1
2,2024-03-27,BRK,WAS,1.645161,2.300000,1
3,2024-03-27,LAC,PHI,1.408163,3.000000,1
4,2024-03-27,NYK,TOR,1.105263,7.250000,1
5,2024-03-27,POR,ATL,5.100000,1.181818,0
6,2024-03-27,LAL,MEM,1.571429,2.450000,1
7,2024-03-27,IND,CHI,1.704225,2.200000,1
8,2024-03-27,HOU,OKC,3.100000,1.387597,0
9,2024-03-27,DET,MIN,9.500000,1.068966,0


In [10]:
makeModelPredictions(games_df, tonight_odds, 'GB')

Predictions made by: GB


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-27,GSW,ORL,2.600000,1.520833,0
1,2024-03-27,CLE,CHO,1.192308,4.900000,1
2,2024-03-27,BRK,WAS,1.645161,2.300000,1
3,2024-03-27,LAC,PHI,1.408163,3.000000,1
4,2024-03-27,NYK,TOR,1.105263,7.250000,0
5,2024-03-27,POR,ATL,5.100000,1.181818,0
6,2024-03-27,LAL,MEM,1.571429,2.450000,1
7,2024-03-27,IND,CHI,1.704225,2.200000,1
8,2024-03-27,HOU,OKC,3.100000,1.387597,0
9,2024-03-27,DET,MIN,9.500000,1.068966,0


In [11]:
makeModelPredictions(games_df, tonight_odds, 'SVM')

Predictions made by: SVM


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-27,GSW,ORL,2.600000,1.520833,1
1,2024-03-27,CLE,CHO,1.192308,4.900000,1
2,2024-03-27,BRK,WAS,1.645161,2.300000,1
3,2024-03-27,LAC,PHI,1.408163,3.000000,0
4,2024-03-27,NYK,TOR,1.105263,7.250000,1
5,2024-03-27,POR,ATL,5.100000,1.181818,0
6,2024-03-27,LAL,MEM,1.571429,2.450000,1
7,2024-03-27,IND,CHI,1.704225,2.200000,1
8,2024-03-27,HOU,OKC,3.100000,1.387597,0
9,2024-03-27,DET,MIN,9.500000,1.068966,0


In [12]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'FCNN')

Predictions made by: FCNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-27,GSW,ORL,2.600000,1.520833,1.0
1,2024-03-27,CLE,CHO,1.192308,4.900000,1.0
2,2024-03-27,BRK,WAS,1.645161,2.300000,1.0
3,2024-03-27,LAC,PHI,1.408163,3.000000,1.0
4,2024-03-27,NYK,TOR,1.105263,7.250000,1.0
5,2024-03-27,POR,ATL,5.100000,1.181818,0.0
6,2024-03-27,LAL,MEM,1.571429,2.450000,1.0
7,2024-03-27,IND,CHI,1.704225,2.200000,1.0
8,2024-03-27,HOU,OKC,3.100000,1.387597,0.0
9,2024-03-27,DET,MIN,9.500000,1.068966,0.0


In [13]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'RNN')

Predictions made by: RNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-27,GSW,ORL,2.600000,1.520833,1.0
1,2024-03-27,CLE,CHO,1.192308,4.900000,1.0
2,2024-03-27,BRK,WAS,1.645161,2.300000,1.0
3,2024-03-27,LAC,PHI,1.408163,3.000000,1.0
4,2024-03-27,NYK,TOR,1.105263,7.250000,1.0
5,2024-03-27,POR,ATL,5.100000,1.181818,0.0
6,2024-03-27,LAL,MEM,1.571429,2.450000,1.0
7,2024-03-27,IND,CHI,1.704225,2.200000,1.0
8,2024-03-27,HOU,OKC,3.100000,1.387597,0.0
9,2024-03-27,DET,MIN,9.500000,1.068966,0.0
